## Building A Chatbot

In [2]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key



'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'

In [4]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000029CBDEAD3C0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000029CBDEADF90>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Arjun and I am a Chief AI Engineer")])

AIMessage(content="Hello Arjun, it's nice to meet you! \n\nThat's a fascinating role.  What kind of AI projects are you working on these days?  \n\nI'm eager to learn more about your work as a Chief AI Engineer.  Perhaps you could tell me about:\n\n* **The industry you work in:** Are you in healthcare, finance, tech, or something else?\n* **Some of the challenges you face:** What are the biggest hurdles in developing and deploying AI solutions?\n* **The most exciting advancements you've seen recently:** What's on the horizon for AI that you're particularly interested in?\n\n\nI'm always learning and I value the opportunity to connect with experts like yourself.  Let's chat!\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 159, 'prompt_tokens': 22, 'total_tokens': 181, 'completion_time': 0.289090909, 'prompt_time': 0.000232829, 'queue_time': 0.012938327999999999, 'total_time': 0.289323738}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf9

In [6]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer"),
        AIMessage(content="Hello Krish! It's nice to meet you. \n\nAs a Chief AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You told me your name is Krish and that you are a Chief AI Engineer.  \n\nIs there anything else you'd like to tell me about yourself or your work? 😊 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 97, 'total_tokens': 137, 'completion_time': 0.072727273, 'prompt_time': 0.002720975, 'queue_time': 0.011385325, 'total_time': 0.075448248}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-86398eb3-3799-4a7e-bd48-81ef067a10b5-0', usage_metadata={'input_tokens': 97, 'output_tokens': 40, 'total_tokens': 137})

In [7]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [8]:
config={"configurable":{"session_id":"chat1"}}

In [11]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Arjun and I am a Chief AI Engineer")],
    config=config
)

In [12]:
response.content

"Hi Arjun, it's great to meet you!\n\nBeing a Chief AI Engineer is a fascinating role. What kind of projects are you currently involved with?  I'm always eager to hear about the innovative work being done in the field of AI. \n\n"

In [13]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='You told me your name is Arjun. 😊  \n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 160, 'total_tokens': 184, 'completion_time': 0.043636364, 'prompt_time': 0.004911353, 'queue_time': 0.008765867, 'total_time': 0.048547717}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-08fc166f-203d-4671-8e51-ab77530096bf-0', usage_metadata={'input_tokens': 160, 'output_tokens': 24, 'total_tokens': 184})

In [14]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'm happy to remember it! 😊\n"

In [15]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hi John, it's nice to meet you!  \n\nIs there anything I can help you with today?  \n\n"

In [16]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"Your name is John, remember?  😄  \n\nI'm happy to help you with anything else you need.  What's on your mind?\n"

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [17]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [18]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Arjun")]})

AIMessage(content="Hello Arjun! It's nice to meet you.  \n\nI'm ready to answer your questions to the best of my ability. What can I help you with today? 😊 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 31, 'total_tokens': 72, 'completion_time': 0.074545455, 'prompt_time': 0.000522588, 'queue_time': 0.013084212000000001, 'total_time': 0.075068043}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-0d894d79-7d4d-493b-a659-8b16763ad656-0', usage_metadata={'input_tokens': 31, 'output_tokens': 41, 'total_tokens': 72})

In [19]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [ ]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Arjun")],
    config=config
)

response

AIMessage(content="Hi Krish, it's nice to meet you! \n\nI'm happy to answer your questions to the best of my ability. What can I help you with today? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 31, 'total_tokens': 73, 'completion_time': 0.076363636, 'prompt_time': 0.000338048, 'queue_time': 0.013341472, 'total_time': 0.076701684}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-63657483-3007-4ea6-b322-fff2b0890625-0', usage_metadata={'input_tokens': 31, 'output_tokens': 42, 'total_tokens': 73})

In [21]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Krish!  I remember that you told me. 😊  Is there anything else I can help you with? \n'

In [22]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [24]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Arjun")],"language":"Hindi"})
response.content

'नमस्ते अर्जुन! मुझे बहुत खुशी है कि आपने मुझसे बात करना चाहा। \n\nआप क्या जानना चाहते हैं? मैं आपकी मदद करने के लिए तैयार हूँ। 😊 \n'

# Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [25]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [ ]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Arjun")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते कृष्ण!  \n\nक्या मैं आपकी मदद कर सकता हूँ? 😊\n'

In [27]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

In [28]:
response.content

'आपका नाम कृष्ण है। 😊 \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [30]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

c:\Users\nagar\Downloads\LLMChatBot\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [31]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As a helpful assistant, I don't have access to your personal information, like your ice cream preferences. \n\nWhat's your favorite flavor? 😊🍦\n"

In [32]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "what is 2 + 2?"  🧮  \n\n\n\nLet me know if you\'d like to try another one!\n'

In [33]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [34]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a helpful assistant, I don't have access to any personal information about you, including your name.  \n\nIs there anything else I can help you with?\n"

In [35]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I have no memory of past conversations.  \n\nIf you have a math problem you'd like help with, please ask! 😄  I'm ready. \n\n"